# mongodb
1. 라이브러리 설치
2. 라이브러리 로드
3. CRUD

In [1]:
!pip install pymongo

     ---------------------------------------- 0.0/382.5 kB ? eta -:--:--
     -------------------- ----------------- 204.8/382.5 kB 6.1 MB/s eta 0:00:01
     -------------------------------------- 382.5/382.5 kB 7.9 MB/s eta 0:00:00
     ---------------------------------------- 0.0/283.7 kB ? eta -:--:--
     -------------------------------------  276.5/283.7 kB 5.7 MB/s eta 0:00:01
     -------------------------------------- 283.7/283.7 kB 4.3 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from pymongo import MongoClient

In [4]:
client = MongoClient(
    host = 'localhost',
    port = 27017
)

In [7]:
# mongodb에 있는 데이터베이스의 목록 출력
client.list_database_names()

['admin', 'config', 'local']

In [6]:
# 데이터베이스를 선택
db = client['ubion']

In [8]:
# collection 접근 (collection은 sql에서 table과 같은 의미)
collection = db['test']

In [10]:
# CRUD -> C
# create -> insert()
# insert_one() / insert_many()
# one은 value와 유사 / many는 vlaues와 유사

data = {
    'id' : 'test',
    'pass' : '12324'
}
collection.insert_one(data)

In [11]:
collection.find_one()
# 입력하지도 않았는데 만들어진 _id 부분의 (32자리의 키)는 절대 중복될 수 없다. primary key와 유사한 역할
# 해쉬값 : 암호화된 데이터로 32 자리. 중복되기 정말 힘들다

{'_id': ObjectId('643f8b0ddbddae7ae9f17f93'), 'id': 'test', 'pass': '12324'}

In [12]:
# 여러 데이터 담는다
data = [
    {
        'name' : 'test1',
        'phone' : '01011112222',
        'gender' : 'male'
    },
    {
        'loc' : 'seoul'
    }
]
collection.insert_many(data)

In [14]:
list(collection.find())

[{'_id': ObjectId('643f8b0ddbddae7ae9f17f93'), 'id': 'test', 'pass': '12324'},
 {'_id': ObjectId('643f8cc5dbddae7ae9f17f94'),
  'name': 'test1',
  'phone': '01011112222',
  'gender': 'male'},
 {'_id': ObjectId('643f8cc5dbddae7ae9f17f95'), 'loc': 'seoul'}]

1. sql에서 test table에서 name이 test1인 경우의 데이터를 출력
    - select * from test where name = 'test1'

In [15]:
list(collection.find({'name' : 'test1'}))

[{'_id': ObjectId('643f8cc5dbddae7ae9f17f94'),
  'name': 'test1',
  'phone': '01011112222',
  'gender': 'male'}]

1. 해당하는 컬럼의 데이터만 출력하려면
    - select name from test

In [17]:
list(collection.find({}, {'name' : 1}))
# {},{'name' : 1} 에서 앞의 소괄호는 where 절 조건, 뒤는 컬럼 조건. 조건이 비어있으면 전체를 뜻한다

[{'_id': ObjectId('643f8b0ddbddae7ae9f17f93')},
 {'_id': ObjectId('643f8cc5dbddae7ae9f17f94'), 'name': 'test1'},
 {'_id': ObjectId('643f8cc5dbddae7ae9f17f95')}]

1. 데이터의 수정
    - update test set pass = '0000' where id = 'test'
    - test 테이블에 id가 test인 데이터에서 pass의 값을 0000으로 수정하겠다

In [20]:
collection.update_one(
    {
        'id' : 'test'
    },
    {
        '$set' : {
            'pass' : '0000'
        }
    }
)

In [21]:
collection.find_one()

{'_id': ObjectId('643f8b0ddbddae7ae9f17f93'), 'id': 'test', 'pass': '0000'}

1. 데이터의 삭제
    - delete from test where id = 'test'
    - test 테이블에서 id가 'test인 데이터를 삭제

In [22]:
collection.delete_one(
    {
    'id' : 'test'
    }
)
# delete_many 하면 test인 값 다 지워짐
# delete_one 하면 상단의 하나만 지워짐

In [24]:
list(collection.find())

[{'_id': ObjectId('643f8cc5dbddae7ae9f17f94'),
  'name': 'test1',
  'phone': '01011112222',
  'gender': 'male'},
 {'_id': ObjectId('643f8cc5dbddae7ae9f17f95'), 'loc': 'seoul'}]

In [25]:
list(client['ubion']['test'].find())
# 딕셔너리에서 key값의 key값을 찾아가는 것과 유사

[{'_id': ObjectId('643f8cc5dbddae7ae9f17f94'),
  'name': 'test1',
  'phone': '01011112222',
  'gender': 'male'},
 {'_id': ObjectId('643f8cc5dbddae7ae9f17f95'), 'loc': 'seoul'}]

### mongodb select 조건식

- find(옵션1, 옵션2, 옵션3, ..., 컬럼 선택)
    
    1. db.collection.find() -> select * from TABLE
    2. db.collection.find({}, {'id':1, 'pass':1}) -> select id, pass from TABLE
    3. db.collection.find({}, {'id':1, 'pass':1, '_id' : 0}) -> 2번과정과 같은 sql문이지만 _id를 보여주지 않겠다.
    4. db.collection.find({'id':'test'}) -> select * from TABLE where id = 'test
    5. db.collection.find({'id':'test', 'pass':'1234'}) -> select * from TABLE where id = 'test' and pass = '1234'
    6. db.collection.find({'id':'test'}, {'pass':'1234'}) -> select * from TABLE where id = 'test' or pass = '1234'
    7. db.collection.find({'age':{'$gt' : 25}}) -> select * from TABLE where age > 25
    8. db.collection.find({'age':{'$lt' : 25}}) -> select * from TABLE where age < 25
    9. db.collection.find({'id':{/bc/}}) -> select * from TABLE where id like '%bc%'
        bc가 포함되어 있는 값
    10. db.collection.find().sort({'id':1}) -> select * from TABLE order by id ASC
    11. db.collection.find().sort({'id':-1}) -> select * from TABLE order by id DESC
    12. db.collection.find().limit(10) -> select * from TABLE limit 10